In [133]:

import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import random
import re
import time
import csv
import math 

import os
import requests
from selenium.webdriver.support.ui import WebDriverWait
from lxml.cssselect import CSSSelector

from tqdm import tqdm


In [3]:
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

#### Get collections info from user collection link 

In [168]:
def get_collection_records(search_link):
    df_tuples_list = []
    request = requests.get(search_link)
    soup = BeautifulSoup(request.content, 'html.parser')
    release_titles = soup.find_all('span', attrs={'class':'release_title'})
    for title in release_titles:
        links_ = title.find_all('a', href=True)
        artist_name = links_[0].get_text()
        artist_link = links_[0]['href']
        release_name = links_[1].get_text()
        release_link = links_[1]['href']
        label_name = links_[2].get_text()
        label_link = links_[2]['href']
        df_tuple = tuple([artist_name, artist_link, release_name, release_link, label_name, label_link])
        df_tuples_list.append(df_tuple)
    return df_tuples_list

In [169]:
test_df = get_collection_records('https://www.discogs.com/user/8markgiam1/collection?page=1&limit=250&header=1')

In [170]:
cols = ['artist_name', 'artist_link', 'release_name', 'release_link', 'label_name', 'label_link']

In [171]:
tdf = pd.DataFrame(test_df, columns=cols)

In [172]:
tdf

,artist_name,artist_link,release_name,release_link,label_name,label_link
0,Babalu And His Headhunters,/artist/5687377-Babalu-And-His-Headhunters,Bahamas Gone Independent / Calypso Funk,/Babalu-And-His-Headhunters-Bahamas-Gone-Indep...,Pressure Makes Diamonds,https://www.discogs.com/label/1664894-Pressure...
1,Red Axes,/artist/2537824-Red-Axes,Voom,/Red-Axes-Voom/release/14497244,Dark Entries,https://www.discogs.com/label/157905-Dark-Entries
2,Nonato & Seu Conjunto*,/artist/989882-Nonato-E-Seu-Conjunto?anv=Nonat...,José Roberto*,/artist/1345894-Zé-Roberto?anv=Jos%C3%A9+Roberto,Cafua / Crioula Multicolorida,/Nonato-Seu-Conjunto-Jos%C3%A9-Roberto-Cafua-C...
3,Antoinette Konan,/artist/3098073-Antoinette-Konan,Antoinette Konan,/Antoinette-Konan-Antoinette-Konan/release/144...,Awesome Tapes From Africa,https://www.discogs.com/label/297290-Awesome-T...
4,Fábio*,/artist/1727403-Fabio-22?anv=F%C3%A1bio,Olhe Fundo Prá Mim (Against All Odds) / Livrem...,/F%C3%A1bio-Olhe-Fundo-Pr%C3%A1-Mim-Against-Al...,EMI,https://www.discogs.com/label/26126-EMI
...,...,...,...,...,...,...
245,Ligabue*,/artist/900001-Luciano-Ligabue?anv=Ligabue,Buon Compleanno Elvis,/Ligabue-Buon-Compleanno-Elvis/release/11662623,Arnoldo Mondadori Editore,https://www.discogs.com/label/189526-Arnoldo-M...
246,Siassia*,/artist/1727918-Albert-Siassia?anv=Siassia,Tokobina,/artist/6127976-Tokobina,Siassia et Tokobina,/Siassia-et-Tokobina-Siassia-et-Tokobina-/rele...
247,Artwork,/artist/66107-Artwork,Let Go Of This Acid,/Artwork-Let-Go-Of-This-Acid/release/10890370,Unfinished Records,https://www.discogs.com/label/1336048-Unfinish...
248,Wolf Müller,/artist/2436644-Wolf-Müller,Niklas Wandt,/artist/2664039-Niklas-Wandt,Instrumentalmusik Von Der Mitte Der World,/Wolf-M%C3%BCller-Niklas-Wandt-Instrumentalmus...


#### iterate that process over all collections pages

Try the data from selected groups (incomplete)


In [4]:
df = pd.read_csv('/Users/regi/Flatiron/final_project/discogs_scrape/selected_groups.csv')

,contributed,for_sale,images,in_collection,in_wantlist,lists,reviews,username
0,/contributions?user=sgt,none,/user/sgt/images,/user/sgt/collection?header=1,none,none,/user/sgt/reviews?header=1,sgt
1,none,/seller/123Jan/profile,none,/user/123Jan/collection?header=1,/wantlist?user=123Jan,none,/user/123Jan/reviews?header=1,123Jan
2,none,none,none,/user/117hao/collection?header=1,/wantlist?user=117hao,none,none,117hao
3,none,none,/user/1111111./images,none,none,none,/user/1111111./reviews?header=1,1111111.
4,/contributions?user=110degreez,none,/user/110degreez/images,/user/110degreez/collection?header=1,none,none,/user/110degreez/reviews?header=1,110degreez
...,...,...,...,...,...,...,...,...
12938,none,none,/user/timcheckley/images,/user/timcheckley/collection?header=1,/wantlist?user=timcheckley,none,none,timcheckley
12939,none,none,none,/user/tropicality/collection?header=1,/wantlist?user=tropicality,none,/user/tropicality/reviews?header=1,tropicality
12940,none,none,none,none,/wantlist?user=TRIPPLE_THR3ATT,none,none,TRIPPLE_THR3ATT
12941,none,/seller/tretno/profile,none,none,none,none,none,tretno


In [7]:
drop = df[(df.in_wantlist == 'none')&(df.in_collection == 'none')].index

In [8]:
len(drop)

3642

In [9]:
df.drop(index=drop, inplace=True)

In [10]:
df

,contributed,for_sale,images,in_collection,in_wantlist,lists,reviews,username
0,/contributions?user=sgt,none,/user/sgt/images,/user/sgt/collection?header=1,none,none,/user/sgt/reviews?header=1,sgt
1,none,/seller/123Jan/profile,none,/user/123Jan/collection?header=1,/wantlist?user=123Jan,none,/user/123Jan/reviews?header=1,123Jan
2,none,none,none,/user/117hao/collection?header=1,/wantlist?user=117hao,none,none,117hao
4,/contributions?user=110degreez,none,/user/110degreez/images,/user/110degreez/collection?header=1,none,none,/user/110degreez/reviews?header=1,110degreez
7,none,/seller/itzmoze/profile,none,/user/itzmoze/collection?header=1,none,none,none,itzmoze
...,...,...,...,...,...,...,...,...
12936,/contributions?user=tintincaipa,none,/user/tintincaipa/images,/user/tintincaipa/collection?header=1,/wantlist?user=tintincaipa,none,/user/tintincaipa/reviews?header=1,tintincaipa
12937,none,none,none,/user/tink01/collection?header=1,none,none,/user/tink01/reviews?header=1,tink01
12938,none,none,/user/timcheckley/images,/user/timcheckley/collection?header=1,/wantlist?user=timcheckley,none,none,timcheckley
12939,none,none,none,/user/tropicality/collection?header=1,/wantlist?user=tropicality,none,/user/tropicality/reviews?header=1,tropicality


In [11]:
user_list = list(set(list(df.username)))

In [12]:
len(user_list)

9301

In [13]:
sample_user = user_list[9]

In [14]:
sample_user

'tmerewether1'

In [15]:
collection_link = 'https://www.discogs.com/user/' + sample_user + '/collection?page=1&limit=250'


In [16]:
wantlist_link = 'https://www.discogs.com/wantlist?page=1&limit=250&user=' + sample_user

In [111]:
import lxml.html
import lxml

In [19]:
response = requests.get(collection_link, stream=True)
response.raw.decode_content = True


In [131]:
users_list = user_list[:10]

In [20]:
tree = lxml.html.parse(response.raw)


In [21]:
from urllib.request import urlopen
from lxml import etree


In [ ]:
ids = re.compile(r'release/(\d*)')

In [169]:
page_number = 1
user_entry = {}
total = re.compile(r'of (.*$)')
ids = re.compile(r'release/(\d*)')

for user in tqdm(users_list):
    release_id_list = []
    collection_link = f'https://www.discogs.com/user/{user}/collection?page={page_number}&limit=250'
    response = urlopen(collection_link)
#     htmlparser = etree.HTMLParser()
    tree = etree.parse(response, etree.HTMLParser())
    total_releses_str = tree.xpath('//strong[@class="pagination_total"]/text()')[0].strip()
#     total_pages = math.ceil(int(total.search(total_releses_str).group(1))/250)
    total_rels = total.search(total_releses_str).group(1)
    if len(total_rels) > 4:
        try:
            total_rels.replace(',','')
        except:
            pass
    total_pages = math.ceil(int(total_rels)/250)
    release_list = tree.xpath('//a[contains(@href, "release") and not(contains(@href, "sell"))]/@href')
    for release in tqdm(release_list):
        try:
            release_id = ids.search(release).group(1)
            release_id_list.append(release_id)
        except:
            pass
    if total_pages > 1:
        for i in tqdm(range(total_pages)):
            collection_link_page = f'https://www.discogs.com/user/{user}/collection?page={i}&limit=250'
            response = urlopen(collection_link)
            htmlparser = etree.HTMLParser()
            tree = etree.parse(response, htmlparser)
            release_list = tree.xpath('//a[contains(@href, "release") and not(contains(@href, "sell"))]/@href')
            for release in release_list:
                try:
                    release_id = ids.search(release).group(1)
                    release_id_list.append(release_id)
                except:
                    pass
    username = tree.getroot().cssselect('.profile_username a')[0].text
    user_entry[username] = release_id_list






  0%|          | 0/10 [00:00<?, ?it/s]





100%|██████████| 263/263 [00:00<00:00, 19749.03it/s][A






  0%|          | 0/2 [00:00<?, ?it/s]





 50%|█████     | 1/2 [00:04<00:04,  4.57s/it]





100%|██████████| 2/2 [00:10<00:00,  5.40s/it]





 10%|█         | 1/10 [00:15<02:20, 15.56s/it]





100%|██████████| 252/252 [00:00<00:00, 14720.14it/s][A






  0%|          | 0/4 [00:00<?, ?it/s]





 25%|██▌       | 1/4 [00:05<00:15,  5.15s/it]





 50%|█████     | 2/4 [00:09<00:09,  5.00s/it]





 75%|███████▌  | 3/4 [00:14<00:04,  4.92s/it]





100%|██████████| 4/4 [00:20<00:00,  5.07s/it]





 20%|██        | 2/10 [00:41<02:29, 18.63s/it]





100%|██████████| 253/253 [00:00<00:00, 14538.42it/s][A






  0%|          | 0/4 [00:00<?, ?it/s]





 25%|██▌       | 1/4 [00:04<00:14,  4.90s/it]





 50%|█████     | 2/4 [00:09<00:09,  4.84s/it]





 75%|███████▌  | 3/4 [00:15<00:05,  5.05s/it]





100%|██████████| 4/4 [00:20<00:00,  5.07s/it]





 30%|███       | 3/10 [01

ValueError: invalid literal for int() with base 10: '4,621'

In [143]:
user_entry.keys()

dict_keys(['prayforthedead', 'Pierre-Alexis', 'mstiefel', 'dRacoir', 'Blank', 'snobo'])

In [144]:
user_entry['mstiefel']

['',
 '119751',
 '643699',
 '42134',
 '113181',
 '70635',
 '167369',
 '6352',
 '69606',
 '32261',
 '5428456',
 '215576',
 '53679',
 '134471',
 '1177823',
 '55115',
 '43315',
 '19723',
 '145811',
 '106504',
 '18116',
 '245663',
 '37360',
 '31076',
 '521188',
 '43541',
 '127461',
 '43636',
 '24174',
 '102881',
 '1639729',
 '802069',
 '4923296',
 '99294',
 '3596665',
 '301976',
 '18816',
 '537378',
 '95056',
 '1298',
 '88',
 '327727',
 '3480',
 '9927',
 '4209',
 '1124',
 '55820',
 '67404',
 '115061',
 '42337',
 '7727804',
 '173286',
 '1182131',
 '82856',
 '1165924',
 '352693',
 '4588833',
 '1097167',
 '12950872',
 '226535',
 '133403',
 '2108345',
 '1292618',
 '1204394',
 '104110',
 '369135',
 '10654',
 '386479',
 '133616',
 '35662',
 '22162',
 '221335',
 '101880',
 '199326',
 '767105',
 '111311',
 '594229',
 '97599',
 '248139',
 '627218',
 '4041853',
 '3481087',
 '132797',
 '43292',
 '47471',
 '32094',
 '198108',
 '18524',
 '939148',
 '92792',
 '1144546',
 '210734',
 '595114',
 '156754',


In [159]:
list(user_entry.items())[0]

('prayforthedead',
 ['',
  '167593',
  '13079875',
  '126197',
  '3371946',
  '4555318',
  '2046325',
  '1981494',
  '2767841',
  '2492862',
  '3464406',
  '2624017',
  '2492104',
  '4946669',
  '447402',
  '1427592',
  '2763973',
  '2640363',
  '12646628',
  '12347431',
  '2164814',
  '224192',
  '1780761',
  '13351761',
  '4719198',
  '5702779',
  '2175183',
  '3508496',
  '414819',
  '9356946',
  '1025404',
  '3505106',
  '7904030',
  '9268847',
  '6404847',
  '11117279',
  '6558835',
  '9113651',
  '606787',
  '521526',
  '3077269',
  '13643463',
  '13643523',
  '6812230',
  '2618486',
  '13477479',
  '13447595',
  '4874347',
  '13348762',
  '13348839',
  '3056899',
  '8338952',
  '12528818',
  '9631387',
  '1543215',
  '7735636',
  '10781824',
  '7641687',
  '1712897',
  '2416464',
  '10408736',
  '6838336',
  '4262248',
  '3657892',
  '12251675',
  '2600619',
  '9780544',
  '10348286',
  '4380676',
  '1552391',
  '590524',
  '2249954',
  '2082590',
  '6284780',
  '4023668',
  '79

In [165]:
pd.DataFrame(data=(list(user_entry.items())), columns=['username', 'collection_releases'])

,username,collection_releases
0,prayforthedead,"[, 167593, 13079875, 126197, 3371946, 4555318,..."
1,Pierre-Alexis,"[, 447603, 81730, 57197, 24469, 1022, 12480752..."
2,mstiefel,"[, 119751, 643699, 42134, 113181, 70635, 16736..."
3,dRacoir,"[, 2095579, 2125667, 2704663, 5951650, 4059834..."
4,Blank,"[, 3172601, ]"
5,snobo,"[, 14652794, 14652622, 11292007, 11264378, 124..."


In [146]:
pd.from_dict(data=user_entry)

AttributeError: module 'pandas' has no attribute 'from_dict'

In [92]:
tree.xpath('//div[@class="profile_username"]/text()')

[]

In [100]:
tree.findall('.profile_username')

[]

In [108]:
css_sel = CSSSelector('.profile_username a')
css_sel.css

'.profile_username a'

In [126]:
el = tree.getroot().cssselect('.profile_username a')[0].text

In [127]:
el

'tmerewether1'

In [125]:
el.text

'tmerewether1'

In [47]:
response = urlopen(collection_link)
htmlparser = etree.HTMLParser()
tree = etree.parse(response, htmlparser)
total_releses_str = tree.xpath('//strong[@class="pagination_total"]/text()')[0].strip()
total_pages = math.ceil(int(total.search(total_releses_str).group(1))/250)
release_list = tree.xpath('//a[contains(@href, "release") and not(contains(@href, "sell"))]/@href')
release_id_list = []
for release in release_list:
    try:
        release_id = ids.search(release).group(1)
        release_id_list.append(release_id)
    except:
        pass

In [ ]:
>>> from lxml.cssselect import CSSSelector


In [48]:
release_id_list

['',
 '54837',
 '29796',
 '10968446',
 '3887702',
 '7896312',
 '10904106',
 '312823',
 '10266584',
 '10181557',
 '9993456',
 '2962613',
 '10003',
 '5683762',
 '3975770',
 '6644667',
 '239193',
 '47391',
 '6341',
 '93311',
 '397126',
 '9476160',
 '7864109',
 '9210429',
 '1734486',
 '9205955',
 '8770614',
 '7994561',
 '1879958',
 '2340176',
 '1232118',
 '2340176',
 '1239176',
 '3810187',
 '726120',
 '8723648',
 '8616525',
 '8281640',
 '8375329',
 '526565',
 '8493060',
 '7296346',
 '6962097',
 '5702908',
 '5725071',
 '1409404',
 '8360498',
 '6882083',
 '7692587',
 '8250360',
 '6364695',
 '5417385',
 '5477268',
 '5512100',
 '8187801',
 '5923809',
 '7047042',
 '5820626',
 '8133875',
 '6612660',
 '8105079',
 '7988040',
 '7235845',
 '13692',
 '7968276',
 '6468787',
 '7974477',
 '2232860',
 '1046469',
 '7854947',
 '7166503',
 '7049399',
 '7055958',
 '173427',
 '1710462',
 '41137',
 '16142',
 '7390839',
 '6588813',
 '7654029',
 '7087425',
 '6641598',
 '7659503',
 '7853834',
 '7785589',
 '755295

In [58]:
total_releses_str = tree.xpath('//strong[@class="pagination_total"]/text()')[0].strip()
total_releses_str

'1 – 148 of 148'

In [55]:
total = re.compile(r'of (.*$)')

In [65]:
total_pages = int(total.search(total_releses_str).group(1))/250
math.ceil(total_pages)

1

In [59]:
total.search(total_releses_str).group(1)

'148'

In [28]:
release_list[3]

'/sell/release/54837?ev=rb'

In [32]:
ids = re.compile(r'release/(\d*)')

In [39]:
release

'/search/?sort=hot%2Cdesc&type=release&ev=em_tr'

In [38]:
ids.search(release).group(1)

AttributeError: 'NoneType' object has no attribute 'group'

In [43]:
release_id_list = []
for release in release_list:
    try:
        release_id = ids.search(release).group(1)
        release_id_list.append(release_id)
    except:
        pass

In [44]:
release_id_list

['',
 '54837',
 '54837',
 '29796',
 '29796',
 '10968446',
 '10968446',
 '3887702',
 '3887702',
 '7896312',
 '7896312',
 '10904106',
 '10904106',
 '312823',
 '312823',
 '10266584',
 '10266584',
 '10181557',
 '10181557',
 '9993456',
 '9993456',
 '2962613',
 '2962613',
 '10003',
 '10003',
 '5683762',
 '5683762',
 '3975770',
 '3975770',
 '6644667',
 '6644667',
 '239193',
 '239193',
 '47391',
 '47391',
 '6341',
 '6341',
 '93311',
 '93311',
 '397126',
 '397126',
 '9476160',
 '9476160',
 '7864109',
 '7864109',
 '9210429',
 '9210429',
 '1734486',
 '9205955',
 '9205955',
 '8770614',
 '8770614',
 '7994561',
 '7994561',
 '1879958',
 '1879958',
 '2340176',
 '2340176',
 '1232118',
 '1232118',
 '2340176',
 '2340176',
 '1239176',
 '1239176',
 '3810187',
 '3810187',
 '726120',
 '726120',
 '8723648',
 '8723648',
 '8616525',
 '8616525',
 '8281640',
 '8281640',
 '8375329',
 '8375329',
 '526565',
 '526565',
 '8493060',
 '8493060',
 '7296346',
 '7296346',
 '6962097',
 '6962097',
 '5702908',
 '5702908',
 '5

In [36]:
ids.search(release_list[3]).group(1)

'54837'

In [ ]:
release_id_list

#### Using selenium chrome driver

In [335]:
group_link = 'https://www.discogs.com/group/96/members'

In [339]:
def get_all_group_pages(main_link):
    all_user_links = []
    #get total users 
    driver = webdriver.Chrome(chromedriver)
    driver.get(main_link)
    total_users = driver.find_element_by_class_name('pagination_total')
    reg_str = 'of (.*$)'
    p = re.compile(reg_str)
    total_str = p.findall(total_users.text)[0]
    total_int = int(total_str.replace(',',''))
    num_of_pages = math.ceil(total_int/250)
    page_links = []
    for page in range(1, num_of_pages + 1):
        page_links.append(f'{main_link}?page={page}&limit=250')
    return page_links
    

In [343]:
def get_all_group_users(page_links):
    page_user_links = []
    for page in page_links:
        driver = webdriver.Chrome(chromedriver)
        driver.get(page)
        link_classes = driver.find_elements_by_class_name('thumbnail_link')
        for link in link_classes:
            user_link = link.get_attribute('href')
            page_user_links.append(user_link)
    return page_user_links

In [341]:
links_96 = get_all_group_pages(group_link)

In [344]:
users_96 = get_all_group_users(links_96)

#### get dict of profile sections

In [363]:
user_link = users_96[0]
user_link

'https://www.discogs.com/user/pinkychukkles'

In [415]:
def get_user_profile_sections(user_link):
    driver = webdriver.Chrome(chromedriver)
    driver.get(user_link)
    profile_sections_element = driver.find_element_by_class_name('profile_sections')
    section_elements = profile_sections_element.find_elements_by_css_selector('li')
    profile_elements_dict = {}
    for element in section_elements:
        elem = element.find_element_by_css_selector('a')
        section_name = elem.text.splitlines()[0]
        profile_elements_dict[section_name] = elem.get_attribute('href')
    return profile_elements_dict

In [418]:
get_user_profile_sections(user_link)

{'For Sale': 'https://www.discogs.com/seller/pinkychukkles/profile',
 'Contributed': 'https://www.discogs.com/contributions?user=pinkychukkles',
 'In Wantlist': 'https://www.discogs.com/wantlist?user=pinkychukkles',
 'Images': 'https://www.discogs.com/user/pinkychukkles/images',
 'Lists': 'https://www.discogs.com/user/pinkychukkles/lists?header=1',
 'Reviews': 'https://www.discogs.com/user/pinkychukkles/reviews?header=1'}

In [421]:
def get_dict_of_user_sections(user_links_list):
    user_sections = {}
    for link in user_links_list:
        user_name = link.split('/')[-1]
        user_sections[user_name] = get_user_profile_sections(link)
    return user_sections

In [423]:
group_96_user_sections = get_dict_of_user_sections(users_96)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".profile_sections"}
  (Session info: chrome=79.0.3945.88)


In [424]:
list1 = ['a', 'b', 'c']

In [425]:
list2 = ['1', '2', '3']

In [429]:
for item in list(zip(list1, list2)):
    if item[0] == 'a':
        print('hm')

hm


In [430]:
lt = list(zip(list1, list2))

In [432]:
lt[0][0]

'a'

In [433]:
dicti = {}

In [434]:
dicti[lt[0][0]] = lt[0][1]

In [435]:
dicti

{'a': '1'}

In [123]:
release_titles = soup.find_all('span', attrs={'class':'release_title'})

In [124]:
tag = release_titles[1]

In [125]:
tag

<span class="release_title set_height">
<a href="/artist/2537824-Red-Axes">Red Axes</a> - Voom (<a href="https://www.discogs.com/label/157905-Dark-Entries">Dark Entries</a> - DE-268)
                                                    <br/><span class="marketplace_for_sale_count">
<a href="/sell/release/14497244?ev=rb"><strong>21 For Sale</strong></a> from <span class="price">$13.00</span></span>
</span>

In [102]:
link.hre

'21 For Sale'

In [107]:
link = tag.find_alal('a', href=True)

In [121]:
link[0]['href']

'/artist/2537824-Red-Axes'

In [142]:
link[0].get_text()

'Red Axes'

In [96]:
type(link.unwrap())

ValueError: Cannot replace an element with its contents when thatelement is not part of a tree.

In [18]:
result_elements = '//span[@class=’release_title’]' #'//input[@id=’user-message’]'

In [22]:
classes = driver.find_elements_by_class_name('release_title')
len(classes)

250

In [51]:
print(classes[5].get_property('text'))

None


In [38]:
print(classes[1].get_attribute('a'))

AttributeError: 'WebElement' object has no attribute 'get_attributes'

In [26]:
for class_ in classes:
    print(class_.get_attribute('href'))
    print(class_)

None
<selenium.webdriver.remote.webelement.WebElement (session="8fa3237c967056a95f3a4eac06c2aa8c", element="877f5c81-6382-4d7b-a251-1e040d2acfa4")>
None
<selenium.webdriver.remote.webelement.WebElement (session="8fa3237c967056a95f3a4eac06c2aa8c", element="62fcda3f-7235-47db-98bb-a8018bce8a58")>
None
<selenium.webdriver.remote.webelement.WebElement (session="8fa3237c967056a95f3a4eac06c2aa8c", element="5fb09336-3f14-4d73-818d-a019eded9161")>
None
<selenium.webdriver.remote.webelement.WebElement (session="8fa3237c967056a95f3a4eac06c2aa8c", element="183e38c1-ffd5-414a-be55-3be60b61d312")>
None
<selenium.webdriver.remote.webelement.WebElement (session="8fa3237c967056a95f3a4eac06c2aa8c", element="eadca7ff-4f5f-4bc1-b80f-ed004a549903")>
None
<selenium.webdriver.remote.webelement.WebElement (session="8fa3237c967056a95f3a4eac06c2aa8c", element="82d76aa5-ed1c-4248-9cd0-4c9d4d405bad")>
None
<selenium.webdriver.remote.webelement.WebElement (session="8fa3237c967056a95f3a4eac06c2aa8c", element="5b2b

None
<selenium.webdriver.remote.webelement.WebElement (session="8fa3237c967056a95f3a4eac06c2aa8c", element="c6ddd104-fa68-4aad-b4da-3d6d4259d254")>
None
<selenium.webdriver.remote.webelement.WebElement (session="8fa3237c967056a95f3a4eac06c2aa8c", element="068c5297-5ad7-41fd-892a-7e5061d7984c")>
None
<selenium.webdriver.remote.webelement.WebElement (session="8fa3237c967056a95f3a4eac06c2aa8c", element="388ca95f-d98e-47fe-93bf-bdf0b0fbbb5d")>
None
<selenium.webdriver.remote.webelement.WebElement (session="8fa3237c967056a95f3a4eac06c2aa8c", element="3da505e9-ba81-49e5-847c-3fed20881ea7")>
None
<selenium.webdriver.remote.webelement.WebElement (session="8fa3237c967056a95f3a4eac06c2aa8c", element="dd5dd4c3-a347-4b95-9a6a-e83081f729d4")>
None
<selenium.webdriver.remote.webelement.WebElement (session="8fa3237c967056a95f3a4eac06c2aa8c", element="8085c6ae-2a13-437d-bdba-56c51e015f98")>
None
<selenium.webdriver.remote.webelement.WebElement (session="8fa3237c967056a95f3a4eac06c2aa8c", element="7e40

None
<selenium.webdriver.remote.webelement.WebElement (session="8fa3237c967056a95f3a4eac06c2aa8c", element="134aacf2-4e32-40c3-aea8-73fde33896a7")>
None
<selenium.webdriver.remote.webelement.WebElement (session="8fa3237c967056a95f3a4eac06c2aa8c", element="35a46beb-38fd-45bf-81b6-f175b994aaa3")>
None
<selenium.webdriver.remote.webelement.WebElement (session="8fa3237c967056a95f3a4eac06c2aa8c", element="65d91119-7ceb-4326-be12-2818e51c2fed")>
None
<selenium.webdriver.remote.webelement.WebElement (session="8fa3237c967056a95f3a4eac06c2aa8c", element="5b891702-e253-44a8-b06e-7b81644f9140")>
None
<selenium.webdriver.remote.webelement.WebElement (session="8fa3237c967056a95f3a4eac06c2aa8c", element="6b0d7b69-a367-4b01-a12d-04e35e1d0a43")>
None
<selenium.webdriver.remote.webelement.WebElement (session="8fa3237c967056a95f3a4eac06c2aa8c", element="30ee01f9-604e-4c1b-b523-9090b47b405e")>
None
<selenium.webdriver.remote.webelement.WebElement (session="8fa3237c967056a95f3a4eac06c2aa8c", element="8ce7

None
<selenium.webdriver.remote.webelement.WebElement (session="8fa3237c967056a95f3a4eac06c2aa8c", element="2b10c15d-d7d5-445f-a3cc-e5fe4a1c26bc")>
None
<selenium.webdriver.remote.webelement.WebElement (session="8fa3237c967056a95f3a4eac06c2aa8c", element="abd82808-5c3b-407b-b09b-c3adc4758b33")>
None
<selenium.webdriver.remote.webelement.WebElement (session="8fa3237c967056a95f3a4eac06c2aa8c", element="5a0f5f75-f3c0-41c6-8ff5-8cb4868be203")>
None
<selenium.webdriver.remote.webelement.WebElement (session="8fa3237c967056a95f3a4eac06c2aa8c", element="62ab213d-af3e-4da0-a629-d6d84eac0f86")>
None
<selenium.webdriver.remote.webelement.WebElement (session="8fa3237c967056a95f3a4eac06c2aa8c", element="90e5509d-c93a-4e56-9948-7e509e3ef5b9")>
None
<selenium.webdriver.remote.webelement.WebElement (session="8fa3237c967056a95f3a4eac06c2aa8c", element="50ef621c-babd-4612-bada-1ec1dd60f815")>
None
<selenium.webdriver.remote.webelement.WebElement (session="8fa3237c967056a95f3a4eac06c2aa8c", element="9213

None
<selenium.webdriver.remote.webelement.WebElement (session="8fa3237c967056a95f3a4eac06c2aa8c", element="12ff57da-bfbd-40ee-abef-d7e9d15596b6")>
None
<selenium.webdriver.remote.webelement.WebElement (session="8fa3237c967056a95f3a4eac06c2aa8c", element="f1d7896e-e827-4b94-8604-07e73c52ce8b")>
None
<selenium.webdriver.remote.webelement.WebElement (session="8fa3237c967056a95f3a4eac06c2aa8c", element="286e5eb3-b664-4e63-8a47-41758eb894b3")>


In [21]:
spans = driver.find_elements_by_tag_name('span')

2724

In [20]:
collections_driver = driver.find_elements_by_xpath('result_elements')
collections_driver

[]

In [ ]:
for url in albumdriver:
            albums.extend([url.get_attribute('href')])

In [ ]:
driver = webdriver.Chrome(chromedriver)
driver.get("https://www.discogs.com/sell/list?sort=listed%2Casc&limit=250&currency=USD&format=Vinyl")

with open('recorddata15.csv', 'w',newline='') as csvfile:
    file = csv.writer(csvfile)
    # make headers
    file.writerow(['Artist_Album', 'Genre', 'Release_Date', 'Style', 'Rate_Haves_Wants', 'Media_Condition',
                     'Sleeve_Condition', 'Seller_Rating', 'Price'])
    count = 381
    
    while count < 501:
        
        # find all links on album marketplace page and store in list

        result_elements = '//a[contains(@href, "/sell/item/")]'

        albums = []

        albumdriver = driver.find_elements_by_xpath(result_elements)

        for url in albumdriver:
            albums.extend([url.get_attribute('href')])
        
        # get rid of duplicates

        albumsclean = [album.split('?', 1)[0] for album in albums]
        albumurls = set(albumsclean)

        # scrape info from album page

        for album in albumurls:

            driver.get(album)

            soup = BeautifulSoup(driver.page_source, 'html.parser')

            try:
                artist_album = driver.find_element_by_xpath("//h1[contains(@id, 'profile_title')]").text.strip()
            except:
                artist_album = np.nan
        
            try:
                genre = driver.find_element_by_xpath("//div[contains(text(), 'Genre:')]/following-sibling::div").text.strip()
            except:
                genre = np.nan

            try:
                release_date = driver.find_element_by_xpath("//div[contains(text(), 'Released:')]/following-sibling::div").text.strip()
            except:
                release_date = np.nan

            try:
                style = driver.find_element_by_xpath("//div[contains(text(), 'Style:')]/following-sibling::div").text.strip()
            except:
                style = np.nan

            try:
                rate_haves_wants = driver.find_element_by_xpath("//a[contains(@class, 'button-blue')]/following-sibling::div").text.strip()
            except:
                rate_haves_wants = np.nan

            try:
                m_condition = driver.find_element_by_xpath("//strong[contains(text(), 'Media:')]/following-sibling::span").text.strip()
            except:
                m_condition = np.nan
    
            try:
                sleeve = driver.find_element_by_xpath("//strong[contains(text(), 'Sleeve:')]")
                s_condition = sleeve.find_element_by_xpath('..').text.strip()
            except:
                s_condition = np.nan

            try:
                seller_rating = driver.find_element_by_xpath("//span[@class='star_rating']/following-sibling::strong").text.strip()
            except:
                seller_rating = np.nan

            try:
                price = soup.find(class_='price').text.strip()
            except:
                price = np.nan

            observation = [artist_album, genre, release_date, style, rate_haves_wants, m_condition, 
                           s_condition, seller_rating, price]

            file.writerow(observation)

            time.sleep(.5+2*random.random())
            
        # go to next page in the marketplace
        
        nextpage = "https://www.discogs.com/sell/list?sort=listed%2Casc&currency=USD&limit=25&page=" + str(count+1) + "&format=Vinyl"

        driver.get(nextpage)
        
        count = count + 1